# Verify the numerical advection-diffusion solver we are using in the Bayesian inverse problem

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Based on the notebook analytical "analytical solution with constant D and varying source.ipynb"
def C_analytical(X, T, Pec, C0=1):
    prefactor = C0 / np.sqrt(4 * np.pi * T / Pec)
    exparg = -Pec * (X - T)**2 / (4 * T)
    return prefactor * np.exp(exparg)

In [ ]:
# imports
import numpy as np
import cuqi
import sys
import matplotlib.pyplot as plt
from cuqi.distribution import Gaussian, JointDistribution
from cuqi.geometry import Continuous2D
from cuqi.pde import TimeDependentLinearPDE
from cuqi.model import PDEModel
import sys
import os
sys.path.append("../ear_aqueducts")
from advection_diffusion_inference_utils import create_domain_geometry, create_PDE_form, build_grids

In [4]:
factor = 3 # factor for refining the grid and the time steps
C0 = 4000
L = 500
n_grid = int(100*factor) # Number of solution nodes
h = L/(n_grid+1) # Space step size
grid = np.linspace(h, L-h, n_grid)

T_0 = 10
tau_max = 30*60 # Final time in sec
cfl = 2 # The cfl condition to have a stable solution
        # the method is implicit, we can choose relatively large time steps 
dt_approx = cfl*h**2 # Defining approximate time step size
n_tau = int((tau_max-T_0)/dt_approx)+1 # Number of time steps

tau = np.linspace(T_0, tau_max, n_tau)

velocity_fd = 2
D_fd = 100
pec_fd = velocity_fd*L/D_fd

In [5]:
# now use the analytical solution to generate the data
u = np.zeros((n_tau, n_grid))

for i, t in enumerate(tau):
    u[i, :] = C_analytical(grid / L, (velocity_fd / L) * (t), pec_fd, C0=C0)

In [ ]:
#%% STEP 4: Create the PDE grid and coefficients grid
#----------------------------------------------------
# PDE and coefficients grids
print(L)
coarsening_factor = 5/factor
n_grid_c = 20
_, grid_c, grid_c_fine, _, _ = build_grids(L, coarsening_factor, n_grid_c)

#%% STEP 5: Create the PDE time steps array
#------------------------------------------
#tau = create_time_steps(h, cfl, tau_max)

#%% STEP 6: Create the domain geometry
#-------------------------------------
G_c = create_domain_geometry(grid_c, "advection_diffusion")

# STEP 7: Create the PDE form
#----------------------------
# c_bc is the analytical solution of the PDE at the boundary x = 0, evaluated
# at tau grid points
c_bc = C_analytical(0, velocity_fd/L*tau, pec_fd, C0=C0)
r_bc = C_analytical(L / L, (velocity_fd / L) * (tau), pec_fd, C0=C0)
u0 = C_analytical(grid / L, (velocity_fd / L) * T_0, pec_fd, C0=C0)
PDE_form = create_PDE_form(c_bc, r_bc, grid, grid_c, grid_c_fine, n_grid, h, tau,
                           "advection_diffusion", u0)

# STEP 8: Create the CUQIpy PDE object
#-------------------------------------
PDE = TimeDependentLinearPDE(PDE_form,
                             tau,
                             grid_sol=grid,
                             method='backward_euler',
                             time_obs='all') 

# STEP 9: Create the range geometry
#----------------------------------
G_cont2D = Continuous2D((grid, tau))

# STEP 10: Create the CUQIpy PDE model
#-------------------------------------
A = PDEModel(PDE, range_geometry=G_cont2D, domain_geometry=G_c)

In [ ]:
# plot lbc and rbc
plt.plot(tau, c_bc, label='lbc')
plt.plot(tau, r_bc, label='rbc')
plt.plot(tau,u[:,-1])
plt.legend()

In [ ]:
print(tau.shape)
print(c_bc.shape)
print(grid_c.shape)
print(G_cont2D.par_dim)


In [9]:
# diffusion paramter is the Peclet number in this case
# Pec = a/(D*L)

# advection parameter is the velocity in this case

exact_x = np.zeros(len(grid_c) +1)
exact_x[:-1] = np.sqrt(D_fd)
exact_x[-1] = velocity_fd
sol = A(exact_x)
#TimeDependentLinearPDE?

In [ ]:
fig, axes = plt.subplots(1, 2, subplot_kw={"projection": "3d"})

plt.sca(axes[0])
xgrid, tgrid = np.meshgrid(grid, tau)

axes[0].plot_surface(
    xgrid[1:, :],
    tgrid[1:, :],
    sol.reshape(( len(grid), len(tau))).T[1:, :]
)

plt.xlabel("distance")
plt.ylabel("time")
axes[0].set_zlabel("concentration")
plt.title(f"numerical Pec = {pec_fd}")


plt.sca(axes[1])


axes[1].plot_surface(
    xgrid[1:, :],
    tgrid[1:, :],
    u[1:, :]
)

plt.xlabel("distance")
plt.ylabel("time")
axes[1].set_zlabel("concentration")
plt.title(f"analytical Pec = {pec_fd}")
plt.show()


# plot as 2D heat map
fig, axes = plt.subplots(1, 2)
plt.sca(axes[0])
plt.pcolor(xgrid[1:, :], tgrid[1:, :], sol.reshape(( len(grid), len(tau))).T[1:, :])
plt.title(f"numerical Pec = {pec_fd}")

plt.colorbar()

plt.sca(axes[1])
plt.pcolor(xgrid[1:, :], tgrid[1:, :], u[1:, :])
plt.title(f"analytical Pec = {pec_fd}")
plt.colorbar()




In [ ]:
# error between the two
print(u[1:,:].shape)
print(sol.reshape(( len(grid), len(tau))).T[1:, :].shape)

error = np.linalg.norm(u[1:,:]-sol.reshape(( len(grid), len(tau))).T[1:, :])
relative_error = error/np.linalg.norm(u[1:,:])

print(f"Error: {error}")
print(f"Relative Error: {relative_error}")